In [ ]:
import chromadb
from pyprojroot import here
from dotenv import load_dotenv
import pandas as pd
import json
import chromadb
load_dotenv()

True

### Create a vector db with all the batch embeddings

In [6]:
chroma_client = chromadb.PersistentClient(path=str(here("data/db/chroma")))
try:
    collection = chroma_client.create_collection(name="imdb")
except:
    collection = chroma_client.get_collection("imdb")

In [7]:

for batch_i in range(1,16):
    embedding = pd.read_csv(here(f'data/embeddings/embeddings_batch_{batch_i}.csv'))
    movie_batch = pd.read_json(here(f'data/movie_batches/movie_batch_{batch_i}.jsonl'), lines=True)
    movie_batch = pd.concat([movie_batch.drop(columns=['body']), pd.json_normalize(movie_batch['body'])], axis=1)
    
    merged = pd.merge(movie_batch, embedding, left_on='custom_id', right_on='custom_id')


    docs = []
    metadatas = []
    ids = []
    embeddings = []

    for index, row in merged.iterrows():
        embedding = json.loads(row['embedding'])
        embeddings.append(embedding)
        docs.append(row['input'])
        metadatas.append({"source": "imdb_movies_100k"})
        ids.append(row['custom_id'])
        
    collection.add(ids=ids, documents=docs, metadatas=metadatas, embeddings=embeddings)
        